In [1]:
import os
import numpy as np
import nibabel as nib
import pandas as pd
import sys
from sklearn import svm
from sklearn.pipeline import Pipeline
from sklearn.model_selection import StratifiedGroupKFold, cross_validate, StratifiedKFold
import numpy as np
from sklearn.linear_model import LogisticRegression
from copy import deepcopy
import itertools

# add path to classification analysis functions
from mixed_sigmoid_normalisation import MixedSigmoidScaler

In [28]:
# Read in SPI directionality info
SPI_directionality_info = pd.read_csv("../feature_extraction/pyspi_SPI_info.csv")

# Load data paths
pyspi_res_path = "/Users/abry4213/data/Cogitate_Batch2/MEG_Data/derivatives/time_series_features"
pyspi_res_path_averaged = f"{pyspi_res_path}/averaged_epochs"
pyspi_res_path_individual = f"{pyspi_res_path}/individual_epochs"

classification_res_path_averaged = "/Users/abry4213/data/Cogitate_Batch2/MEG_Data/derivatives/classification_results/across_participants"
classification_res_path_individual = "/Users/abry4213/data/Cogitate_Batch2/MEG_Data/derivatives/classification_results/within_participants"

Classification within individual subjects

In [ ]:
# Load in pyspi results
for participant_pyspi_res_file in os.listdir(pyspi_res_path_individual):
    individual_subject_pyspi_res = pd.read_csv(f"{pyspi_res_path_individual}/{participant_pyspi_res_file}")

    # Extract subject ID
    subject_ID = participant_pyspi_res_file.split("_")[0]

    # Fix stimulus_type where False to 'false' 
    individual_subject_pyspi_res['stimulus_type'] = individual_subject_pyspi_res['stimulus_type'].replace(False, 'false')

    # meta-ROI comparisons: GWNT --> CS, CS --> GNWT, IIT --> CS, CS --> IIT
    meta_roi_comparisons = [("GNWT", "Category_Selective"), ("Category_Selective", "GNWT"), ("IIT", "Category_Selective"), ("Category_Selective", "IIT")]

    # Relevance type comparisons
    relevance_type_comparisons = ["Relevant non-target", "Irrelevant"]

    # Stimulus presentation comparisons
    stimulus_presentation_comparisons = individual_subject_pyspi_res.stimulus_presentation.unique().tolist()

    # Stimulus type comparisons
    stimulus_types = individual_subject_pyspi_res.stimulus_type.unique().tolist()
    stimulus_type_comparisons = list(itertools.combinations(stimulus_types, 2))

    # All comparisons list
    n_jobs = 1
    comparing_between_stimulus_types_classification_results_list = []

    for meta_roi_comparison in meta_roi_comparisons:
        print("ROI Comparison:" + str(meta_roi_comparison))
        ROI_from, ROI_to = meta_roi_comparison
        for relevance_type in relevance_type_comparisons:
            print("Relevance type:" + str(relevance_type))
            for stimulus_presentation in stimulus_presentation_comparisons:
                print("Stimulus presentation:" + str(stimulus_presentation))
                # Finally, we get to the final dataset
                final_dataset_for_classification = individual_subject_pyspi_res.query("meta_ROI_from == @ROI_from & meta_ROI_to == @ROI_to & relevance_type == @relevance_type & stimulus_presentation == @stimulus_presentation").reset_index(drop=True).drop(columns=['index'])

                for SPI in final_dataset_for_classification.SPI.unique():

                    # Extract this SPI
                    this_SPI_data = final_dataset_for_classification.query(f"SPI == '{SPI}'")

                    # Find overall number of rows
                    num_rows = this_SPI_data.shape[0]

                    # Extract SPI values
                    this_column_data = this_SPI_data["value"]

                    # Find number of NaN in this column 
                    num_NaN = this_column_data.isna().sum()
                    prop_NaN = num_NaN / num_rows

                    # Find mode and SD
                    column_mode_max = this_column_data.value_counts().max()
                    column_SD = this_column_data.std()

                    # If 0% < num_NaN < 10%, impute by the mean of each component
                    if 0 < prop_NaN < 0.1:
                        values_imputed = (this_column_data
                                            .transform(lambda x: x.fillna(x.mean())))

                        this_column_data = values_imputed
                        print(f"Imputing column values for {SPI}")
                        this_SPI_data["value"] = this_column_data

                    # If there are: 
                    # - more than 10% NaN values;
                    # - more than 90% of the values are the same; OR
                    # - the standard deviation is less than 1*10**(-10)
                    # then remove the column
                    if prop_NaN > 0.1 or column_mode_max / num_rows > 0.9 or column_SD < 1*10**(-10):
                        print(f"{SPI} has low SD: {column_SD}, and/or too many mode occurences: {column_mode_max} out of {num_rows}, and/or {100*prop_NaN}% NaN")
                        continue
                    
                    # Start an empty list for the classification results
                    SPI_combo_res_list = []
                
                    # Iterate over stimulus combos
                    for this_combo in stimulus_type_comparisons:

                        # Extract just GNWT/CS data first
                        final_dataset_for_classification_this_combo = this_SPI_data.query(f"stimulus_type in {this_combo}")

                        # Define classification model
                        model = LogisticRegression(penalty='l1', C=1, solver='liblinear', random_state=127)
                        pipe = Pipeline([('scaler', MixedSigmoidScaler(unit_variance=True)), 
                                                ('model', model)])

                        # Fit classifier
                        X = final_dataset_for_classification_this_combo.value.to_numpy().reshape(-1, 1)
                        y = final_dataset_for_classification_this_combo.stimulus_type.to_numpy().reshape(-1, 1)

                        stimulus_stratified_CV = StratifiedKFold(n_splits = 10, shuffle = True, random_state=127)

                        this_classifier_res = cross_validate(pipe, X, y, cv=stimulus_stratified_CV, scoring="accuracy", n_jobs=n_jobs, 
                                                                    return_estimator=False, return_train_score=False)["test_score"].mean()
                        
                        this_SPI_combo_df = pd.DataFrame({"SPI": [SPI], 
                                                            "meta_ROI_from": [ROI_from],
                                                            "meta_ROI_to": [ROI_to],
                                                            "relevance_type": [relevance_type],
                                                            "stimulus_presentation": [stimulus_presentation],
                                                            "stimulus_combo": [this_combo], 
                                                            "accuracy": [this_classifier_res]})
                        
                        # Append to growing results list
                        comparing_between_stimulus_types_classification_results_list.append(this_SPI_combo_df)

    comparing_between_stimulus_types_classification_results = pd.concat(comparing_between_stimulus_types_classification_results_list).reset_index(drop=True)
    comparing_between_stimulus_types_classification_results.to_csv(f"{classification_res_path_individual}/{subject_ID}_comparing_between_stimulus_types_classification_results.csv", index=False)

In [8]:
comparing_between_stimulus_types_classification_results.to_csv(f"{classification_res_path_individual}/{subject_ID}_comparing_between_stimulus_types_classification_results.csv", index=False)

Then, we can ask how -- for each stimulus type -- each SPI distinguishes between task relevance vs. irrelevance

In [37]:
# All comparisons list
comparing_between_relevance_types_classification_results_list = []

meta_roi_comparison = meta_roi_comparisons[0]
ROI_from, ROI_to = meta_roi_comparison
stimulus_type = stimulus_types[0]
stimulus_presentation = stimulus_presentation_comparisons[0]
SPI = "anm"

# Finally, we get to the final dataset
final_dataset_for_classification = all_pyspi_res.query("meta_ROI_from == @ROI_from & meta_ROI_to == @ROI_to & stimulus_type == @stimulus_type & stimulus == @stimulus_presentation & relevance_type in @relevance_type_comparisons").reset_index(drop=True).drop(columns=['index'])


# Extract this SPI
this_SPI_data = final_dataset_for_classification.query(f"SPI == '{SPI}'")

# Look up directionality
this_SPI_directionality = SPI_directionality_info.query(f"SPI == '{SPI}'").Directionality.values[0]

# Merge meta ROIs according to directionality
if this_SPI_directionality == "Directed":
    this_SPI_data["meta_ROI_pair"] = this_SPI_data.meta_ROI_from + "_" + this_SPI_data.meta_ROI_to
    this_SPI_data = this_SPI_data.drop(["meta_ROI_from", "meta_ROI_to"], axis=1)
else:
    this_SPI_data_sorted = [sorted(pair) for pair in this_SPI_data[["meta_ROI_from", "meta_ROI_to"]].values.tolist()]
    this_SPI_data['meta_ROI_pair'] = ['_'.join(string) for string in this_SPI_data_sorted]
    this_SPI_data = (this_SPI_data
                .drop(["meta_ROI_from", "meta_ROI_to"], axis=1)
                .drop_duplicates(ignore_index=True, subset=['subject_ID', 'meta_ROI_pair', 'relevance_type', 'stimulus'])
                )

# Find overall number of rows
num_rows = this_SPI_data.shape[0]

# Extract SPI values
this_column_data = this_SPI_data["value"]

# Find number of NaN in this column 
num_NaN = this_column_data.isna().sum()
prop_NaN = num_NaN / num_rows

# Find mode and SD
column_mode_max = this_column_data.value_counts().max()
column_SD = this_column_data.std()

# If 0% < num_NaN < 10%, impute by the mean of each component
if 0 < prop_NaN < 0.1:
    values_imputed = (this_column_data
                        .transform(lambda x: x.fillna(x.mean())))

    this_column_data = values_imputed
    print(f"Imputing column values for {SPI}")
    this_SPI_data["value"] = this_column_data

# If there are: 
# - more than 10% NaN values;
# - more than 90% of the values are the same; OR
# - the standard deviation is less than 1*10**(-10)
# then remove the column
if prop_NaN > 0.1 or column_mode_max / num_rows > 0.9 or column_SD < 1*10**(-10):
    print(f"{SPI} has low SD: {column_SD}, and/or too many mode occurences: {column_mode_max} out of {num_rows}, and/or {100*prop_NaN}% NaN")

# Start an empty list for the classification results
SPI_combo_res_list = []

# Define classification model
model = LogisticRegression(penalty='l1', C=1, solver='liblinear', random_state=127)
pipe = Pipeline([('scaler', MixedSigmoidScaler(unit_variance=True)), 
                        ('model', model)])

# Fit classifier
X = this_SPI_data.value.to_numpy().reshape(-1, 1)
y = this_SPI_data.relevance_type.to_numpy().reshape(-1, 1)
groups = this_SPI_data.subject_ID.to_numpy().reshape(-1, 1)

group_stratified_CV = StratifiedGroupKFold(n_splits = 10, shuffle = True, random_state=127)

this_classifier_res = cross_validate(pipe, X, y, groups=groups, cv=group_stratified_CV, scoring="accuracy", n_jobs=1, 
                                            return_estimator=False, return_train_score=False)["test_score"].mean()

this_SPI_relevance_results_df = pd.DataFrame({"SPI": [SPI], 
                                    "meta_ROI_from": [ROI_from],
                                    "meta_ROI_to": [ROI_to],
                                    "stimulus_type": [stimulus_type],
                                    "stimulus_presentation": [stimulus_presentation],
                                    "comparison": ["Relevant non-target vs. Irrelevant"], 
                                    "accuracy": [this_classifier_res]})

# Append to growing results list
comparing_between_stimulus_types_classification_results_list.append(this_SPI_relevance_results_df)

/var/folders/rn/z0683sns3tz7b_k0cbf8ft8w0000gq/T/ipykernel_93316/1934161159.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  this_SPI_data["meta_ROI_pair"] = this_SPI_data.meta_ROI_from + "_" + this_SPI_data.meta_ROI_to
/Users/abry4213/anaconda3/envs/annie_env/lib/python3.9/site-packages/sklearn/utils/validation.py:1229: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/abry4213/anaconda3/envs/annie_env/lib/python3.9/site-packages/sklearn/utils/validation.py:1229: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for exam

In [31]:
SPI = "cohmag_multitaper_max_fs-1_fmin-0-25_fmax-0-5"
ROI_from = "IIT" 
ROI_to = "Category_Selective"
stimulus_type = "object"
stimulus_presentation = "on"

# Extract this SPI
this_SPI_data = final_dataset_for_classification.query(f"SPI == '{SPI}'")

# Find overall number of rows
num_rows = this_SPI_data.shape[0]

# Extract SPI values
this_column_data = this_SPI_data["value"]

# Find number of NaN in this column 
num_NaN = this_column_data.isna().sum()
prop_NaN = num_NaN / num_rows

# Find mode and SD
column_mode_max = this_column_data.value_counts().max()
column_SD = this_column_data.std()

# If 0% < num_NaN < 10%, impute by the mean of each component
if 0 < prop_NaN < 0.1:
    values_imputed = (this_column_data
                        .transform(lambda x: x.fillna(x.mean())))

    this_column_data = values_imputed
    print(f"Imputing column values for {SPI}")
    this_SPI_data["value"] = this_column_data

# If there are: 
# - more than 10% NaN values;
# - more than 90% of the values are the same; OR
# - the standard deviation is less than 1*10**(-10)
# then remove the column
if prop_NaN > 0.1 or column_mode_max / num_rows > 0.9 or column_SD < 1*10**(-10):
    print(f"{SPI} has low SD: {column_SD}, and/or too many mode occurences: {column_mode_max} out of {num_rows}, and/or {100*prop_NaN}% NaN")

# Start an empty list for the classification results
SPI_combo_res_list = []

# Define classification model
model = LogisticRegression(penalty='l1', C=1, solver='liblinear', random_state=127)
pipe = Pipeline([('scaler', MixedSigmoidScaler(unit_variance=True)), 
                        ('model', model)])

# Fit classifier
X = this_SPI_data.value.to_numpy().reshape(-1, 1)
y = this_SPI_data.relevance_type.to_numpy().reshape(-1, 1)
groups = this_SPI_data.subject_ID.to_numpy().reshape(-1, 1)

group_stratified_CV = StratifiedGroupKFold(n_splits = 10, shuffle = True, random_state=127)

this_classifier_res = cross_validate(pipe, X, y, groups=groups, cv=group_stratified_CV, scoring="accuracy", n_jobs=1, 
                                            return_estimator=False, return_train_score=False)["test_score"]



/Users/abry4213/anaconda3/envs/annie_env/lib/python3.9/site-packages/sklearn/utils/validation.py:1229: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/abry4213/anaconda3/envs/annie_env/lib/python3.9/site-packages/sklearn/utils/validation.py:1229: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/abry4213/anaconda3/envs/annie_env/lib/python3.9/site-packages/sklearn/utils/validation.py:1229: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/abry4213/anaconda3/envs/annie_env/lib/python3.9/site-packages/sklearn/utils/validation.py:1229: DataConversionWarning

In [32]:
this_classifier_res.mean()

0.47750000000000004